<a href="https://colab.research.google.com/github/2303A51553/Natural-language-process/blob/main/2303A51553_Legal_document_Summarisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow nltk pandas numpy sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.5 MB/s eta 0:00:00


In [2]:
import re
import string
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = str(text).lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in stop_words]
    return " ".join(tokens)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
nltk.download('punkt_tab')
df = pd.read_csv("/content/Legal_Summarisation_100_Final.csv")

df["clean_doc"] = df["document"].apply(preprocess_text)
df["clean_sum"] = df["summary"].apply(preprocess_text)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [5]:
BOS = "[BOS]"
EOS = "[EOS]"

df["clean_sum"] = df["clean_sum"].apply(lambda x: f"{BOS} {x} {EOS}")


In [6]:
max_vocab = 25000
max_enc_len = 256
max_dec_len = 64

vectorizer = layers.TextVectorization(
    max_tokens=max_vocab,
    output_mode="int",
    output_sequence_length=max_dec_len,
    standardize=None
)

# Adapt vocabulary
vectorizer.adapt(df["clean_doc"].tolist() + df["clean_sum"].tolist())

# Get vocab
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)

# Token IDs
BOS_ID = vocab.index(BOS)
EOS_ID = vocab.index(EOS)
PAD_ID = 0


In [7]:
enc = vectorizer(df["clean_doc"].tolist()).numpy()
dec = vectorizer(df["clean_sum"].tolist()).numpy()

# Decoder input starts with BOS
dec_in = np.concatenate([np.full((dec.shape[0],1), BOS_ID), dec[:,:-1]], axis=1)

# Decoder target is full summary
dec_out = dec


In [8]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, vocab_size, embed_dim, max_len):
        super().__init__()
        self.token_emb=layers.Embedding(vocab_size, embed_dim, mask_zero=True)
        self.pos_emb=layers.Embedding(max_len, embed_dim)

    def call(self, x):
        positions=tf.range(start=0, limit=tf.shape(x)[1])
        pos_embeddings=self.pos_emb(positions)
        tok_embeddings=self.token_emb(x)
        return tok_embeddings+pos_embeddings


In [9]:
def bert_encoder_layer(embed_dim, num_heads, ff_dim):
    inp=layers.Input(shape=(None, embed_dim))
    attn=layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(inp, inp)
    out1=layers.LayerNormalization()(inp + attn)

    ffn=layers.Dense(ff_dim, activation="relu")(out1)
    ffn=layers.Dense(embed_dim)(ffn)
    out2=layers.LayerNormalization()(out1 + ffn)

    return keras.Model(inp, out2)


In [10]:
def bert_decoder_layer(embed_dim, num_heads, ff_dim):
    dec_in=layers.Input(shape=(None, embed_dim))
    enc_out=layers.Input(shape=(None, embed_dim))

    # Masked self-attention
    attn1=layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_dim, use_causal_mask=True
    )(dec_in, dec_in)
    out1=layers.LayerNormalization()(dec_in + attn1)

    # Cross-attention
    attn2=layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_dim
    )(out1, enc_out)
    out2=layers.LayerNormalization()(out1 + attn2)

    # Feed-forward
    ffn=layers.Dense(ff_dim, activation="relu")(out2)
    ffn=layers.Dense(embed_dim)(ffn)
    out3=layers.LayerNormalization()(out2 + ffn)

    return keras.Model([dec_in, enc_out],out3)


In [21]:
embed_dim=256
ff_dim=512
num_heads=8
num_layers=4

def bert_decoder_layer(embed_dim, num_heads, ff_dim):
    dec_in=layers.Input(shape=(None, embed_dim))
    enc_out=layers.Input(shape=(None, embed_dim))

    # Instantiate MultiHeadAttention layers
    mha_self_attn = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
    mha_cross_attn = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)

    # Masked self-attention: pass use_causal_mask=True to the call method
    attn1=mha_self_attn(query=dec_in, value=dec_in, use_causal_mask=True)
    out1=layers.LayerNormalization()(dec_in + attn1)

    # Cross-attention
    attn2=mha_cross_attn(query=out1, value=enc_out, key=enc_out)
    out2=layers.LayerNormalization()(out1 + attn2)

    # Feed-forward
    ffn=layers.Dense(ff_dim, activation="relu")(out2)
    ffn=layers.Dense(embed_dim)(ffn)
    out3=layers.LayerNormalization()(out2 + ffn)

    return keras.Model([dec_in, enc_out],out3)

# Encoder
enc_input = keras.Input(shape=(None,), dtype="int32")
enc_emb = PositionalEmbedding(vocab_size, embed_dim, max_enc_len)(enc_input)
encoder = enc_emb
for _ in range(num_layers):
    encoder = bert_encoder_layer(embed_dim, num_heads, ff_dim)(encoder)

# Decoder
dec_input=keras.Input(shape=(None,),dtype="int32")
dec_emb=PositionalEmbedding(vocab_size,embed_dim, max_dec_len)(dec_input)
decoder=dec_emb
for _ in range(num_layers):
    decoder=bert_decoder_layer(embed_dim, num_heads,ff_dim)([decoder,encoder])

# Output
final_output=layers.Dense(vocab_size)(decoder)

model=keras.Model([enc_input,dec_input],final_output)
model.summary()

Model: "functional_37"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_54      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 256) │     89,088 │ input_layer_54[0… │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_29       │ (None, None, 256) │  2,367,488 │ positional_embed… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_30       │ (None, None, 256) │  2,367,488 │ functional_29[0]… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_59      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_31       │ (None, None, 256) │  2,367,488 │ functional_30[0]… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 256) │     39,936 │ input_layer_59[0… │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_32       │ (None, None, 256) │  2,367,488 │ functional_31[0]… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_33       │ (None, None, 256) │  4,471,552 │ positional_embed… │
│ (Functional)        │                   │            │ functional_32[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_34       │ (None, None, 256) │  4,471,552 │ functional_33[0]… │
│ (Functional)        │                   │            │ functional_32[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_35       │ (None, None, 256) │  4,471,552 │ functional_34[0]… │
│ (Functional)        │                   │            │ functional_32[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_36       │ (None, None, 256) │  4,471,552 │ functional_35[0]… │
│ (Functional)        │                   │            │ functional_32[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_73 (Dense)    │ (None, None, 92)  │     23,644 │ functional_36[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 27,508,828 (104.94 MB)

 Trainable params: 27,508,828 (104.94 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer="adam",loss=loss_fn)

model.fit([enc, dec_in],dec_out,batch_size=4,epochs=6)


Epoch 1/6
25/25 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - loss: 2.6583
Epoch 2/6
25/25 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step - loss: 0.8239
Epoch 3/6
25/25 ━━━━━━━━━━━━━━━━━━━━ 102s 3s/step - loss: 0.8138
Epoch 4/6
25/25 ━━━━━━━━━━━━━━━━━━━━ 79s 3s/step - loss: 0.8160
Epoch 5/6
25/25 ━━━━━━━━━━━━━━━━━━━━ 72s 3s/step - loss: 0.8068
Epoch 6/6
25/25 ━━━━━━━━━━━━━━━━━━━━ 64s 3s/step - loss: 0.8049


In [18]:
id_to_word={i:w for i,w in enumerate(vocab)}

def generate(text, max_len=60):
    e=vectorizer([text]).numpy()
    d=np.zeros((1, max_len), dtype=int)
    d[0,0]=BOS_ID

    for i in range(1, max_len):
        preds=model.predict([e, d[:,:i]],verbose=0)
        next_id=np.argmax(preds[0, i-1])

        if next_id==EOS_ID:
            break

        d[0,i]=next_id

    tokens=[id_to_word[t] for t in d[0] if t not in [PAD_ID, BOS_ID, EOS_ID]]
    return " ".join(tokens)


In [22]:
import sacrebleu

preds=[]
refs=[]

for i in range(30):
    p=generate(df["clean_doc"].iloc[i])
    preds.append(p)
    refs.append(df["clean_sum"].iloc[i])

bleu=sacrebleu.corpus_bleu(preds, [refs])
print("BLEU Score:", bleu.score)


BLEU Score: 0.046290446176410774
